<a href="https://colab.research.google.com/github/dennisbonesso/DIO_sqlAlchemy/blob/main/sqlalchemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import Column
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import ForeignKey

In [2]:
Base = declarative_base()

In [3]:
class User(Base):
      __tablename__ = "user_account"
      id = Column(Integer,primary_key=True)
      name = Column(String)
      fullname = Column(String)

      adress = relationship(
          "Address", back_populates="user",cascade="all, delete-orphan"
      )

      def __repr__(self):
          return f"User(id={self.id},name={self.name},fullname={self.fullname})"



In [4]:
class Address(Base):
      __tablename__ = "address"
      id = Column(Integer,primary_key=True)
      email_adress = Column(String(30),nullable=False)
      user_id=Column(Integer,ForeignKey("user_account.id"),nullable=False)

      user = relationship("User",back_populates="address")

      def __repr__(self):
          return f"Address(id={self.id},email={self.email_address})"

In [6]:
print(User.__tablename__)
print(Address.__tablename__)

user_account
address
